In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
import tables
import flammkuchen as fl
from pathlib import Path
from tqdm import tqdm
from fimpylab.utils.suite2p_conv import export_suite2p
import tifffile as tiff
import matplotlib.pyplot as plt

from fimpylab.core.lightsheet_experiment import LightsheetExperiment

import json
from scipy.optimize import quadratic_assignment
from lotr.pca import qap_sorting_and_phase

import statsmodels.api as sm

In [ ]:
master_path =  Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v29_29p1s\ls\1sec")
files = list(master_path.glob("*_f*"))

print(len(files))

In [ ]:
for f in files[:2]:
    fish_dir = f
    print(fish_dir)
    suite2p_data = fl.load(fish_dir / 'data_from_suite2p_cells.h5')
    
    norm_traces = traces = fl.load(fish_dir / 'filtered_traces.h5')['detr']
    num_rois = np.shape(norm_traces)[0]
    norm_traces = norm_traces.T
    
    corrmat = np.corrcoef(norm_traces)
    ng_corr_ind = np.unique(np.where(corrmat < -0.6)[0])
    corrmat_neg = corrmat[ng_corr_ind]
    norm_traces_neg = norm_traces[ng_corr_ind]

    num_ng = np.shape(ng_corr_ind)[0]
    print(ng_corr_ind)
    
    perm, com_phase = qap_sorting_and_phase(norm_traces_neg.T)
    sorted_traces = norm_traces_neg[perm]
    sorted_corrmat = np.corrcoef(sorted_traces)
    
    '''
    flow = np.zeros((n_traces, n_traces))
    toshift = np.cos(np.linspace(-np.pi, np.pi, n))
    for i in range(n):
        flow[i, :] = np.roll(toshift, i)
        
    options = {"P0": "randomized"}
    res = min([soo.quadratic_assignment(flow, corrmat_neg, method="faq", options = options) 
                 for i in range(1000)], key=lambda x: x.fun)

    options = {"partial_guess": np.array([np.arange(n), res.col_ind]).T}
    res = soo.quadratic_assignment(flow, corrmat_neg, method="2opt", options=options)

    perm = res["col_ind"]
    '''
    print(np.shape(com_phase), np.shape(perm))
    anatomy = np.sum(suite2p_data['anatomy_stack'], 0)
    anatomy = np.rot90(anatomy, k=3)
    
    fig_all, ax = plt.subplots(2, 3, figsize=(15,10))
    ax[0,0].imshow(anatomy, cmap="gray_r")
    ax[0,0].axis('off')
    ax[0,0].set_title('Anatomy')
    
    ax[0,1].imshow(norm_traces_neg, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-2, vmax=2)
    ax[0,1].axis('off')
    ax[0,1].set_title('Negatively correlated traces')
    
    ax[0,2].imshow(sorted_traces, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-2, vmax=2)
    ax[0,2].axis('off')
    ax[0,2].set_title('Sorted traces')
    
    rois_full = suite2p_data['rois_stack']
    for i in range(8):
        rois = rois_full[i]
        rois = np.rot90(rois, k=3)
        rois = np.ma.masked_where(rois < 1, rois)
        ax[1,0].imshow(rois, cmap="rainbow", alpha=0.5)
    ax[1,0].axis('off')
    ax[1,0].set_title('ROIs')
                   
    ax[1,1].imshow(sorted_corrmat.T, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-1, vmax=1)
    ax[1,1].axis('off')
    ax[1,1].set_title('Sorted corrmat')
    
    rois_sorted_full = np.zeros_like(rois_full)
    roi_color = np.arange(num_ng)[perm]
    count=0
    for i in ng_corr_ind:
        rois_sorted_full[np.where(rois_full == (i))] = roi_color[count] + 1
        count +=1
        
    num_ng = np.shape(ng_corr_ind)[0]
    print(num_ng)
    for i in range(8):
        rois_sorted = rois_sorted_full[i]
        rois_sorted = np.rot90(rois_sorted, k=3)
        rois_sorted = np.ma.masked_where(rois_sorted < 1, rois_sorted)
        rois_sorted[i,0:num_ng] = np.arange(num_ng)
        print(np.max(rois_sorted))
        ax[1,2].imshow(rois_sorted, cmap=COLS['phase'], alpha=1)
    #rois_sorted = np.ma.masked_where(rois_sorted < 1, rois_sorted)
    #ax[1,2].imshow(rois_sorted, cmap="twilight")
    ax[1,2].axis('off')
    ax[1,2].set_title('ROIs (colored by ind)')
    
    file_name = 'quad_sorted_traces_and_corrmat_thresh07_filtered.jpg'
    fig_all.savefig(fish_dir / file_name, dpi=300)

In [ ]:
fish_dir = files[4]
suite2p_data = fl.load(fish_dir / 'data_from_suite2p_cells.h5')
roi_coords = suite2p_data['coords']

norm_traces = traces = fl.load(fish_dir / 'filtered_traces.h5')['detr']
num_rois = np.shape(norm_traces)[0]
norm_traces = norm_traces.T

corrmat = np.corrcoef(norm_traces)
ng_corr_ind = np.unique(np.where(corrmat < -0.55)[0])
corrmat_neg = corrmat[ng_corr_ind]
norm_traces_neg = norm_traces[ng_corr_ind]

num_ng = np.shape(ng_corr_ind)[0]

min_val = np.nanmin(corrmat, axis=0)

corrmat2 = np.zeros_like(corrmat)
corrmat2[np.where(corrmat < -0.55)] = 1
num_min_per_cell= np.sum(corrmat2, axis=1)


anatomy = np.sum(suite2p_data['anatomy_stack'], 0)
anatomy = np.rot90(anatomy, k=3)

fig_all, ax = plt.subplots(3, 3, figsize=(8,12), sharex=True, sharey=True)
ax[0,0].imshow(anatomy, cmap="gray_r")
ax[0,0].axis('off')
ax[0,0].set_title('Anatomy')

ax[0,1].scatter(roi_coords[:, 1], roi_coords[:, 2], c=min_val, cmap='hot', s=3, vmin=-1, vmax=0, alpha=0.5)
ax[0,1].axis('off')
ax[0,1].set_title('Minimal correlation')

ax[0,2].scatter(roi_coords[ng_corr_ind, 1], roi_coords[ng_corr_ind, 2], c='k', s=3, alpha=0.5)
ax[0,2].axis('off')
ax[0,2].set_title('Minimal correlated neurons')

ax[1,0].scatter(roi_coords[ng_corr_ind, 1], roi_coords[ng_corr_ind, 2], c=num_min_per_cell[ng_corr_ind], cmap='Greens', s=3, vmin=0, vmax=20, alpha=0.5)
ax[1,0].axis('off')
ax[1,0].set_title('Number of antcorrelated neurons')




n_lags = 1000
norm_traces_neg_for_ac = norm_traces_neg.T
num_traces_ng = np.shape(ng_corr_ind)[0]
AC_all = np.zeros((n_lags+1, num_traces_ng))
for i in range(num_traces_ng):
    ys =  sm.tsa.acf(norm_traces_neg_for_ac[:,i], nlags=n_lags)
    AC_all[:, i] = ys
    
ac_tau = AC_all[100, :]
ax[1,1].scatter(roi_coords[ng_corr_ind, 1], roi_coords[ng_corr_ind, 2], c=ac_tau, cmap='Reds', s=3)
ax[1,1].axis('off')
ax[1,1].set_title('Autocorrelation at 100')

ac_tau = AC_all[900, :]
ax[1,2].scatter(roi_coords[ng_corr_ind, 1], roi_coords[ng_corr_ind, 2], c=ac_tau, cmap='Reds', s=3)
ax[1,2].axis('off')
ax[1,2].set_title('Autocorrelation at 500')


reg_values = fl.load(fish_dir / 'motor_regressors.h5')
print(reg_values.keys())

reg_right = reg_values['right_bias']
thresh_ind = np.where(abs(reg_right) > 0.5)[0]
ax[2,0].scatter(roi_coords[thresh_ind, 1], roi_coords[thresh_ind, 2], c=reg_right[thresh_ind], cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax[2,0].axis('off')
ax[2,0].set_title('Right bias')

reg_left = reg_values['left_bias'] 
thresh_ind = np.where(abs(reg_left) > 0.5)[0]
ax[2,1].scatter(roi_coords[thresh_ind, 1], roi_coords[thresh_ind, 2], c=reg_left[thresh_ind], cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax[2,1].axis('off')
ax[2,1].set_title('Left bias')


reg_left = reg_values['left_bias'] 
thresh_ind = np.where(abs(reg_left) > 0.5)[0]
ax[2,2].scatter(roi_coords[thresh_ind, 1], roi_coords[thresh_ind, 2], c=reg_left[thresh_ind], cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax[2,2].axis('off')
ax[2,2].set_title('Heading')

'''

n_lags = 1000
norm_traces_neg_for_ac = norm_traces.T
num_traces = np.shape(norm_traces)[0]
AC_all = np.zeros((n_lags+1, num_traces))
for i in range(num_traces):
    ys =  sm.tsa.acf(norm_traces_neg_for_ac[:,i], nlags=n_lags)
    AC_all[:, i] = ys
    
ac_tau = AC_all[100, :]
ax[1,1].scatter(roi_coords[:, 1], roi_coords[:, 2], c=ac_tau, cmap='Reds', s=3)
ax[1,1].axis('off')
ax[1,1].set_title('Autocorrelation at 100')

ac_tau = AC_all[900, :]
ax[1,2].scatter(roi_coords[:, 1], roi_coords[:, 2], c=ac_tau, cmap='Reds', s=3)
ax[1,2].axis('off')
ax[1,2].set_title('Autocorrelation at 500')



rois_sorted_full = np.zeros_like(rois_full)
roi_color = np.arange(num_ng)[perm]
count=0
for i in ng_corr_ind:
    rois_sorted_full[np.where(rois_full == (i))] = roi_color[count] + 1
    count +=1

num_ng = np.shape(ng_corr_ind)[0]
print(num_ng)
for i in range(8):
    rois_sorted = rois_sorted_full[i]
    rois_sorted = np.rot90(rois_sorted, k=3)
    rois_sorted = np.ma.masked_where(rois_sorted < 1, rois_sorted)
    rois_sorted[i,0:num_ng] = np.arange(num_ng)
    print(np.max(rois_sorted))
    ax[1,2].imshow(rois_sorted, cmap=COLS['phase'], alpha=1)
#rois_sorted = np.ma.masked_where(rois_sorted < 1, rois_sorted)
#ax[1,2].imshow(rois_sorted, cmap="twilight")
ax[1,2].axis('off')
ax[1,2].set_title('ROIs (colored by ind)')

'''
file_name = 'ROI_maps_220302.jpg'
fig_all.savefig(fish_dir / file_name, dpi=300)


In [ ]:
np.unique(num_min_per_cell)
np.shape(num_min_per_cell)


In [ ]:
for f in files:
    fish_dir = f
    print(fish_dir)
    suite2p_data = fl.load(fish_dir / 'data_from_suite2p_cells.h5')
    
    norm_traces = traces = fl.load(fish_dir / 'filtered_traces.h5')['detr']
    num_rois = np.shape(norm_traces)[0]
    norm_traces = norm_traces.T
    '''

    corrmat = np.corrcoef(norm_traces)
    ng_corr_ind = np.unique(np.where(corrmat < -0.6)[0])
    corrmat_neg = corrmat[ng_corr_ind]
    norm_traces_neg = norm_traces[ng_corr_ind]

    num_ng = np.shape(ng_corr_ind)[0]
    print(ng_corr_ind)
    '''

    fig_all, ax = plt.subplots(1, 2, figsize=(6,6), gridspec_kw={'height_ratios': [1, 3]})
    ax[0].imshow(anatomy, cmap="gray_r")
    ax[0].axis('off')
    ax[0].set_title('Tail')

    ax[1].imshow(norm_traces, extent=[0, 500, 0, 500], cmap="coolwarm", vmin=-2, vmax=2)
    ax[1].axis('off')
    ax[1].set_title('Normalized traces')

    file_name = 'tail_and_norm_traces.jpg'
    fig_all.savefig(fish_dir / file_name, dpi=300)